## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
"""import zipfile

file_path = '/content/'

output_unzip = zipfile.ZipFile("/content/drive/MyDrive/project/BreastCancer/breast_bags_256.zip", "r")  # "r": read 모드
output_unzip.extractall(file_path)
output_unzip.close()

"""

NotImplementedError: ignored

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':88
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
!git clone https://github.com/rosasalberto/mil.git
!pip install mil
!pip install numpy
!pip install scikit-learn
!pip install scipy
!pip install tensorflow

Cloning into 'mil'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 183 (delta 5), reused 3 (delta 0), pack-reused 165
Receiving objects: 100% (183/183), 9.95 MiB | 12.74 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 21.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## usage

In [13]:
from mil.data.datasets import musk1
from mil.data.datasets import mnist_bags

# importing bag_representation
from mil.bag_representation import MILESMapping
# importing validation strategy
from mil.validators import LeaveOneOut
# importing final model, which in this case is the SVC classifier from sklearn
from mil.models import SVC
# importing trainer
from mil.trainer import Trainer
# importing preprocessing 
from mil.preprocessing import StandarizerBagsList
# importing metrics, which in this case are from tf keras metrics
from mil.metrics import AUC



In [14]:
# loading dataset
(bags_train, y_train), (bags_test, y_test) = mnist_bags.load()


FileNotFoundError: ignored

In [15]:
len(bags_train),len(y_train),len(bags_test),len(y_test)


NameError: ignored

In [ ]:
y_train

array([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1], dtype=int64)

In [ ]:
print(bags_train[7]),len(bags_train[7])

[[42, -14, 16, -44, -117, 41, -164, 59, -219, -26, -109, -83, -59, -88, -82, -284, -111, -17, -59, -34, -105, -185, -202, 146, 79, -213, 75, -104, -64, 100, -117, -104, -39, 25, -151, 155, -157, -52, 5, 35, 20, -26, -57, -106, -109, -189, 14, -120, 11, -21, -94, -200, -202, -1, 140, -124, -79, 42, -22, 9, -97, -113, 19, 110, -135, 79, -165, 34, 71, 67, 22, -40, -71, -71, -18, -174, -218, -44, -90, 10, -165, -175, -126, 138, 38, -198, 97, -125, -45, 125, -169, 20, -106, -101, -56, 213, -48, -174, -13, -97, 10, 52, -65, -121, -115, -46, -162, -142, -1, -120, -60, -155, -192, -198, 47, 11, 159, -136, -136, 61, 77, 84, 97, 50, -94, 62, -45, 86, -64, -188, -70, -79, -63, -57, -35, 70, -23, 118, -213, -112, -135, -25, -136, -165, -178, -102, -124, 91, 98, 67, -110, -125, -110, -50, -79, -80, -234, -175, 67, -15, 128, 103, 69, -38, -34, -46], [32, -104, -123, 109, -117, 55, 47, 8, -37, -78, -44, -24, -27, -124, -97, -288, 59, -52, -25, -62, -66, -17, 14, -66, -102, -52, 69, -119, 41, -136, 34

(None, 4)

In [ ]:
print(bags_train[7][1]), len(bags_train[7][1])

[32, -104, -123, 109, -117, 55, 47, 8, -37, -78, -44, -24, -27, -124, -97, -288, 59, -52, -25, -62, -66, -17, 14, -66, -102, -52, 69, -119, 41, -136, 34, 130, 83, -183, 38, 103, 135, 7, -143, 73, -128, 69, -39, -81, -88, -107, -84, -92, -47, -89, -69, -19, 1, 65, 17, -37, 100, -129, 66, -135, 197, 85, -17, -143, 105, 59, -165, -41, -133, 1, -136, -138, -30, -65, -45, -179, 2, -89, -14, 55, -36, 10, -46, -42, -163, -2, -44, -138, 29, -123, 44, 111, 136, -89, 0, 74, 111, 19, -153, 94, -114, 122, -46, -56, -114, -130, -143, -82, -71, -101, -59, -4, 23, 29, -178, -76, -101, -3, 75, 34, -117, -29, -90, 95, 115, 46, -95, -159, -76, 96, 24, 54, -90, -75, -106, -95, 42, -82, -20, -60, -57, 42, 63, 15, -178, -102, -124, -11, -32, 82, 45, 68, 67, -106, 117, 23, -227, -229, -182, -91, -103, 79, 118, -21, -22, 114]


(None, 166)

In [ ]:
from PIL import Image
import numpy as np

array = bags_train[7][2]

array = np.array(array)

In [ ]:
type(array), array.shape

(numpy.ndarray, (166,))

In [ ]:
array=array.reshape(83,2)

In [ ]:
import numpy as np
from PIL import Image


pil_image=Image.fromarray(array)
pil_image.show()


In [ ]:
# instantiate trainer
trainer = Trainer()

# preparing trainer
metrics = ['acc', AUC]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]
trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)

# fitting trainer
valid = LeaveOneOut()
history = trainer.fit(bags_train, y_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

# printing validation results for each fold
print(history['metrics_val'])

# predicting metrics for the test set
trainer.predict_metrics(bags_test, y_test)


73/73 [==============================] - 11s 154ms/step - train_accuracy: 1.0000 - train_auc: 1.0000 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00
[{'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy':

{'accuracy': 0.68421054, 'auc': 0.72727275}